### Preprocessing the input data

In [4]:
import numpy as np
import tensorflow as tf
import keras

ERROR! Session/line number was not unique in database. History logging moved to new session 41


In [ ]:
import tf.keras.utils.image_dataset_from_directory

batch_size = 16
img_height = 224
img_width = 224

data_dir = 'data/train'

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.1,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.1,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

### MobileNetV2

In [ ]:
# Downloading the model
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.preprocessing.image import ImageDataGenerator

model = MobileNetV2(weights='imagenet')

### Customizing the model

In [ ]:

from keras import Model
from keras.layers import Dense
import keras

corn_output = Dense(5, activation='softmax')
corn_output = corn_output(model.layers[-2].output)
corn_input = model.input
corn_model = Model(inputs=corn_input, outputs=corn_output)

for layer in corn_model.layers[:-1]:
    layer.trainable

corn_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='acc',
        patience=10,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath='my_model.h5',
        monitor='val_loss',
        save_best_only=True,
    ),
    tf.keras.callbacks.CSVLogger(
        "Log", 
        separator=",", 
        append=False
    )
]

corn_model.fit(
    train_ds,
    epochs=100,
    verbose=2,
    validation_data=val_ds,
    callbacks=callbacks_list
)

model.save('corn_classification.h5')